In [1]:
import warnings

import torch

from omnisafe.common.experiment_grid import ExperimentGrid
import omnisafe

In [5]:
eg = ExperimentGrid(exp_name='BM_Safety_CPO')

second_order_policy = ['CPO']

safety_envs = [
    'SafetyPointGoal1-v0',
]
eg.add('env_id', safety_envs)

avaliable_gpus = list(range(torch.cuda.device_count()))
gpu_id = [0]

if gpu_id and not set(gpu_id).issubset(avaliable_gpus):
    warnings.warn('The GPU ID is not available, use CPU instead.', stacklevel=1)
    gpu_id = None

eg.add('algo', second_order_policy)
eg.add('logger_cfgs:use_wandb', [True])
eg.add('logger_cfgs:wandb_project', ['Benchmark_Safety'])
eg.add('train_cfgs:vector_env_nums', [4])
eg.add('train_cfgs:torch_threads', [8])
eg.add('algo_cfgs:steps_per_epoch', [20000])
eg.add('train_cfgs:total_steps', [20000 * 30])
eg.add('seed', [0])

In [6]:
"""Example of training a policy from exp-x config with OmniSafe."""
import os, sys
from omnisafe.typing import NamedTuple, Tuple


def train(
    exp_id: str, algo: str, env_id: str, custom_cfgs: NamedTuple
) -> Tuple[float, float, float]:
    """Train a policy from exp-x config with OmniSafe.

    Args:
        exp_id (str): Experiment ID.
        algo (str): Algorithm to train.
        env_id (str): The name of test environment.
        custom_cfgs (NamedTuple): Custom configurations.
        num_threads (int, optional): Number of threads. Defaults to 6.
    """
    terminal_log_name = 'terminal.log'
    error_log_name = 'error.log'
    if 'seed' in custom_cfgs:
        terminal_log_name = f'seed{custom_cfgs["seed"]}_{terminal_log_name}'
        error_log_name = f'seed{custom_cfgs["seed"]}_{error_log_name}'
    sys.stdout = sys.__stdout__
    sys.stderr = sys.__stderr__
    print(f'exp-x: {exp_id} is training...')
    if not os.path.exists(custom_cfgs['logger_cfgs']['log_dir']):
        os.makedirs(custom_cfgs['logger_cfgs']['log_dir'], exist_ok=True)
    # pylint: disable-next=consider-using-with
    sys.stdout = open(
        os.path.join(f'{custom_cfgs["logger_cfgs"]["log_dir"]}', terminal_log_name),
        'w',
        encoding='utf-8',
    )
    # pylint: disable-next=consider-using-with
    sys.stderr = open(
        os.path.join(f'{custom_cfgs["logger_cfgs"]["log_dir"]}', error_log_name),
        'w',
        encoding='utf-8',
    )
    agent = omnisafe.Agent(algo, env_id, custom_cfgs=custom_cfgs)
    reward, cost, ep_len = agent.learn()
    return reward, cost, ep_len

In [7]:
# total experiment num must can be divided by num_pool
# meanwhile, users should decide this value according to their machine
eg.run(train, gpu_id=gpu_id)

Save with config of experiment grid in grid_config.json

ExperimentGrid [BM_Safety_CPO] runs over parameters:

 env_id                                  

[env] 

	"SafetyPointGoal1-v0"



 algo                                    

...

Warning: trajectory cut off when rollout by epoch at 1000.0 steps.

Alert! Attempting infeasible recovery!

Expected Improvement: -0.0038872621953487396 Actual: -0.003883539466187358

Accept step at i=1

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Metrics                       ┃ Value                  ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Metrics/EpRet                 │ 9.423103332519531      │
│ Metrics/EpCost                │ 31.479999542236328     │
│ Metrics/EpLen                 │ 1000.0                 │
│ Train/Epoch                   │ 29.0                   │
│ Train/Entropy                 │ 1.358576774597168      │
│ Train/KL                      │ 0.00016258397954516113 │
│ Train/StopIter                │ 10.0                   │
│ Train/PolicyRatio/Mean        │ 0.9999920725822449     │
│ Train/PolicyRatio/Min         │ 0.9999920725822449     │
│ Train/PolicyRatio/Max         │ 0.9999920725822449     │
│ Train/PolicyRatio/Std         │ 5.619614967145026e-06  │
│ Train/LR                      │ 0.0                    │
│ Train/PolicyStd               │ 0.9519319534301758     │
│ TotalEnvSteps                 │ 600000.0               │
│ Loss/Loss_pi                  │ -0.0042234305292367935 │
│ Loss/Loss_pi/Delta            │ 0.0036050863564014435  │
│ Value/Adv                     │ 6.866454960174906e-09  │
│ Loss/Loss_reward_critic       │ 0.017556460574269295   │
│ Loss/Loss_reward_critic/Delta │ 0.0003144461661577225  │
│ Value/reward                  │ 0.6362228393554688     │
│ Loss/Loss_cost_critic         │ 0.8488020896911621     │
│ Loss/Loss_cost_critic/Delta   │ -0.19485211372375488   │
│ Value/cost                    │ 2.7300522327423096     │
│ Time/Total                    │ 1831.47705078125       │
│ Time/Rollout                  │ 49.22683334350586      │
│ Time/Update                   │ 13.330299377441406     │
│ Time/Epoch                    │ 62.557132720947266     │
│ Time/FPS                      │ 319.707763671875       │
│ Misc/Alpha                    │ 0.9522688388824463     │
│ Misc/FinalStepNorm            │ 0.2926810085773468     │
│ Misc/gradient_norm            │ 0.1454121470451355     │
│ Misc/xHx                      │ 0.022055182605981827   │
│ Misc/H_inv_g                  │ 0.24867244064807892    │
│ Misc/AcceptanceStep           │ 1.0                    │
│ Misc/cost_gradient_norm       │ 0.16049514710903168    │
│ Misc/A                        │ 0.021299641579389572   │
│ Misc/B                        │ -944.3695068359375     │
│ Misc/q                        │ 0.022055182605981827   │
│ Misc/r                        │ 0.005795998033136129   │
│ Misc/s                        │ 0.04446299374103546    │
│ Misc/Lambda_star              │ 0.0                    │
│ Misc/Nu_star                  │ 0.6706803441047668     │
│ Misc/OptimCase                │ 0.0                    │
└───────────────────────────────┴────────────────────────┘

In [8]:
LOG_DIR = 'exp-x\BM_Safety_CPO\SafetyPointGoal1-v0---a666ef11f41efb3aebf109cd70c4d4f6f3fe654ce38f5144dfb4fe07a0c7c165\CPO-{SafetyPointGoal1-v0}\seed-000-2024-04-24-11-29-43'

evaluator = omnisafe.Evaluator()
for item in os.scandir(os.path.join(LOG_DIR, 'torch_save')):
    if item.is_file() and item.name.split('.')[-1] == 'pt':
        evaluator.load_saved(
            save_dir=LOG_DIR, model_name=item.name, camera_name='track', width=256, height=256
        )
        evaluator.render(num_episodes=5)
        evaluator.evaluate(num_episodes=5)